# Artificial Neural Network

### Importing the libraries

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf

- Here `tensorflow 2.0` is already pre-installed on google colab.

In [26]:
tf.__version__

'2.7.0'

## Part 1 - Data Preprocessing

### Importing the dataset

- Dataset information : 
  - `RowNumber` : serial number
  - `CustomerId` : customer's ID
  - `Surname ` : Surname of customer
  - `CreditScore` : Credit Score
  - `Geography` : Where's the customer from
  - `Gender` : Gender of Customer
  - `Age` : Age of Customer
  - `Tenure` : The number of years since the customer is in the bank
  - `Balance  ` : balance in account
  - `NumOfProducts` : The number of products they use from bank
  - `HasCrCard` : Has Credit card?
  - `IsActiveMember` : is Active member
  - `EstimatedSalary` : Estimated Salary
  - `Exited` : does leaved the bank


In [27]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values   # excluding roll-numbers, customer-IDs and Surname
y = dataset.iloc[:,-1].values

- Also here, neural network can figure out that it doesn't need roll numbers, customer IDs etc. but still 😅

In [28]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [29]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [30]:
print(y)

[1 0 1 ... 1 1 0]


- No missing data in the dataset

### Encoding categorical data

Label Encoding the "Gender" column

In [31]:
print('Original : ', X[1])

Original :  [608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]


In [32]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [33]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [34]:
X[1]

array([608, 'Spain', 0, 41, 1, 83807.86, 1, 0, 1, 112542.58], dtype=object)

One Hot Encoding the "Geography" column
- As there is no co-relation b/w different countries. and ther are more then 2 countries included here so it's better to go for `OneHotEncoding` than `LabelEncoder`. 
- we'll use LabelEncoder in `gender` column as there are only two values.

In [35]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [36]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


- All generated columns are moved at beggining

In [37]:
X[1]

array([0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
      dtype=object)

### Splitting the dataset into the Training set and Test set

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

### Feature Scaling
- **NOTE** : 
  - Feature Scalling is compulsory in Deep Learning
  - No matter what are values of our columns are. but we have to apply feature scalling to them too. *ALL MEANS ALL*

In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)   # only transforming to avoid Information Leakage

## Part 2 - Building the ANN

### Initializing the ANN
- Here `Sequential` is from `keras` library. and it's now integrated in `Tensorflow` library.

In [40]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer
- `units` : Number of neurons in the layer.
  > Here there is no rule of thumb to decide how to choose optimal number of units. this can only be chosen by experimenting with different values.
- `relu` : **Rectifier** Activation Function 


In [41]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer
- Here adding second layer will be no different process then the first one. although,  
  > we can make some tweaks in that too!

In [42]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer
- Here dimention of the output `y` is 1, so we'll choose the number of neurons as 1.
- Here as activation function, we are choosing SIGMA as we have binary classification.

In [43]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN
- Updating weights and reducing cost.
- `optimizer` : gradient descent method. Here we are choosing `adam` as one of the method.
- `loss` : as we are doing binary classification, we'll use `binary_crossentropy` which will apply `CROSS ENTROPY` eq.
  > For non-binary classification use `categorical_crossentropy` as `loss` function and `softmax` as `activation` function above.
- `metrics` : choosing accuracy. although we can also choose multiple metrices.
 

In [44]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set
- we need two more arguments : `batch_size` & `epochs` rather than conventional method of only giving `X_train` and `y_train`.
- Here we are also setting the `batch size` which means instead of checking predictions and comparing it with every example, we'll compare it after these number of examples.

In [45]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6019 - accuracy: 0.6629
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4805 - accuracy: 0.7985
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4483 - accuracy: 0.8069
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4346 - accuracy: 0.8126
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.8188
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4112 - accuracy: 0.8274
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3979 - accuracy: 0.8361
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3859 - accuracy: 0.8420
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3756 - accuracy: 0.8454
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3672 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [70]:
# [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58]
# [0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58]
# CreditScore,	Geography,	Gender,	Age,	Tenure,	Balance,	NumOfProducts,	HasCrCard,	IsActiveMember,	EstimatedSalary,	Exited
test_case = np.array([[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000]])
test_case[:,2] = le.transform(test_case[:,2])
print('After LabelEncoding : ',test_case)
test_case = ct.transform(test_case)
print('After OneHotEncoding : ',test_case)

test_case = sc.transform(test_case)

# Only if we are not feature scaling the test case:
# test_case = [[float(i) for i in test_case[0]]]    # Have to manually convert all str items to float.
# print('After converting string values to float : ',test_case)

print('Prediction : ',ann.predict(test_case))
print('Should we say goodbye to this customer ? :','YES' if ann.predict([test_case])>0.5 else 'NO')

After LabelEncoding :  [['600' 'France' '1' '40' '3' '60000' '2' '1' '1' '50000']]
After OneHotEncoding :  [['1.0' '0.0' '0.0' '600' '1' '40' '3' '60000' '2' '1' '1' '50000']]
Prediction :  [[0.02650735]]
Should we say goodbye to this customer ? : NO


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [78]:
y_pred = (ann.predict(X_test)>0.5)

# y_pred = ann.predict(X_test)
# y_pred = (y_pred>0.5)
# print(y_pred)

print(
      np.concatenate(
          (y_pred.reshape(len(y_pred),1) , y_test.reshape(len(y_test),1)),1
      )
     )

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [83]:
from sklearn.metrics import confusion_matrix, accuracy_score
print('Confusion Matrix :\n', confusion_matrix(y_test, y_pred))
print('Accuracy Score :', accuracy_score(y_test, y_pred))

Confusion Matrix :
 [[1526   69]
 [ 201  204]]
Accuracy Score : 0.865
